In [85]:
from os import listdir
from os.path import isfile, join
import random
from PIL import Image

import numpy as np
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf


Learning_Rate=1e-5
width=height=80 # image width and height
batchSize=4096

In [86]:
torch.cuda.is_available()

True

In [87]:
path = 'GeneratedImages/' + files[random.randrange(0,len(files))]
print(path)
s = path.split('_')[2]
s = s[0:-16]
s

GeneratedImages/edited1_frame0_108.30083565459609degrees_x-1_y-9crop.jpg


'108.30083'

In [88]:
path = 'GeneratedImages'
files = [f for f in listdir(path) if isfile(join(path, f))]
len(files)

144001

In [89]:
def LoadRandomImg():
    path = 'GeneratedImages/' + files[random.randrange(0,len(files))]
    image = Image.open(path)
    
    s = path.split('_')[2]
    angle = s[0:-7]

    transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)),tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) 
    image=transformImg(np.array(image))
    return image, float(angle)

In [90]:
def LoadBatch():
    images = torch.zeros([batchSize,3,height,width])
    angles = torch.zeros([batchSize])
    for i in range(batchSize):
        images[i],angles[i]=LoadRandomImg()
    return images,angles

In [91]:
#--------------Load and set net and optimizer-------------------------------------

# Set device GPU or CPU where the training will take place
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

#load net
Net = torchvision.models.resnet18(pretrained=True)

# Change final layer to predict one value
Net.fc = torch.nn.Linear(in_features=512, out_features=1, bias=True) 

Net = Net.to(device)

# Create adam optimizer
optimizer = torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) 



#----------------Train------------------------------------------------------------

# Save average loss for display
AverageLoss=np.zeros([50]) 

# Training loop
for itr in range(500001): 
    # Load taining batch
    images,GTFillLevel=LoadBatch()

    # Load image
    images=torch.autograd.Variable(images,requires_grad=False).to(device) 
    
    # Load Ground truth fill level
    GTFillLevel = torch.autograd.Variable(GTFillLevel, requires_grad=False).to(device) 
    
    # make prediction
    PredLevel=Net(images)
    Net.zero_grad()
    Loss=torch.abs(PredLevel-GTFillLevel).mean()
    
    # Backpropogate loss
    Loss.backward() 
    
    # Apply gradient descent change to weight
    optimizer.step() 
    
    # Save loss average
    AverageLoss[itr%50]=Loss.data.cpu().numpy() 
    
    print(itr,") Loss=",Loss.data.cpu().numpy(),'AverageLoss',AverageLoss.mean()) # Display loss
    
    # Save model
    if itr % 200 == 0: 
        #Save model weight
        print("Saving Model" +str(itr) + ".torch") 
        torch.save(Net.state_dict(),   str(itr) + ".torch")

0 ) Loss= 80.85829 AverageLoss 1.6171658325195313
Saving Model0.torch
1 ) Loss= 79.71418 AverageLoss 3.2114494323730467
2 ) Loss= 80.71149 AverageLoss 4.825679168701172
3 ) Loss= 82.60982 AverageLoss 6.477875518798828
4 ) Loss= 80.09138 AverageLoss 8.079703063964844
5 ) Loss= 80.73312 AverageLoss 9.69436553955078
6 ) Loss= 80.57937 AverageLoss 11.305952911376954


KeyboardInterrupt: 